In [3]:
!pip install langchain pypdf


  Using cached pypdf-4.2.0-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.2.0-py3-none-any.whl (290 kB)


In [4]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("D20200321Lj.pdf")
pages = loader.load()

In [3]:
!pip install PyPDF2

589.70s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [6]:
import PyPDF2
import re

In [12]:
def extract_text_from_pdf(pdf_path):
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text_by_page = []

    # Extract text from each page
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        text_by_page.append(text)
    
    return text_by_page

def remove_header_footer(text_by_page):
    def clean_first_page_of_legal_act(text):
        return re.split("(?=U\s*S\s*T\s*A\s*W\s*A)", text)[-1]
    def remove_footer_from_page(text):
        split_point = list(re.finditer("\\n\s+\\n", text))
        if split_point:
            last_split = split_point[-1].end()
            text = text[:last_split].strip()
        return text    
        
        

    cleaned_text_by_page = []
    # Regex pattern to match the header/footer with flexible date
    pattern = 4

    for i in range(0,len(text_by_page)):
        
        cleaned_text = re.sub("^©\s*Kancelaria\s+Sejmu\s*s\.\s+\d+/\d+\s*\\n\s*\\n\d{4}\s*[-]?\s*\d{2}\s*[-]?\s*\d{2}", '', text_by_page[i]).strip()
        
        if i == 0:
            cleaned_text = clean_first_page_of_legal_act(cleaned_text)
            # cleaned_text = re.split("(U\s*S\s*T\s*A\s*W\s*A\s*).*", cleaned_text)
        r =4
        cleaned_text = remove_footer_from_page(cleaned_text)
        cleaned_text_by_page.append(cleaned_text)

    # for page_text in text_by_page:
    #     parts = re.split("^©\s*Kancelaria\s+Sejmu\s*s\.\s+\d+/\d+\s*\\n\s*\\n\d{4}\s*[-]?\s*\d{2}\s*[-]?\s*\d{2}", page_text)
    #     cleaned_text = re.sub("^©\s*Kancelaria\s+Sejmu\s*s\.\s+\d+/\d+\s*\\n\s*\\n\d{4}\s*[-]?\s*\d{2}\s*[-]?\s*\d{2}", '', page_text).strip()
    #     cleaned_text_by_page.append(cleaned_text)
    
    return cleaned_text_by_page
def concatenate_pages(cleaned_text_by_page):
    full_text = ""
    for page in cleaned_text_by_page:
        full_text += page + " "
    return full_text
def remove_new_line(cleaned_text):
    return cleaned_text.replace('\n', ' ')

text_by_page = extract_text_from_pdf("D20200321Lj.pdf")
cleaned_text_by_page = remove_header_footer(text_by_page)
cleaned_legal_act = concatenate_pages(cleaned_text_by_page)
f = 4
cleaned_legal_act = remove_new_line(cleaned_legal_act)

In [5]:
full_text = ""
for page in pages:
    print(page.page_content)
    full_text += page.page_content + "\n"

©Kancelaria Sejmu     s. 1/7 
 
2023 -08-03 
  
Dz. U. 2020 poz. 321   
 
 
U S T AWA  
z dnia 9  stycznia 2020  r. 
o dodatkowym rocznym świadczeniu pieniężnym dla emerytów i  rencistów  
Art. 1. Ustawa określa warunki nabywania prawa do dodatkowego rocznego 
świadczenia pieniężnego oraz zasady jego wypłaty.  
Art. 2. 1. Dodatkowe roczne świadczenie pieniężne przysługuje osobom, 
które w  dniu 31  marca roku, w  którym wyp łacane jest dodatkowe roczne 
świadczenie pieniężne, mają prawo do:  
1) świadczeń, o  których mowa w  art. 2 pkt 1 lit. a i b ustawy z  dnia 29  maja 
1974  r. o zaopatrzeniu inwalidów wojennych i  wojskowyc h oraz ich rodzin 
(Dz. U. z 2022  r. poz.  2287),  
2) świadcz eń z  ubezpieczenia emerytalno -rentowego, o  których mowa 
w art. 18 pkt 1–4 ustawy z  dnia 20  grudnia 1990  r. o  ubezpieczeniu 
społecznym rolników (Dz.  U. z 2022 r. poz. 933, 1155 i 2140),  
3) świadczeń, o  których mowa w  art. 12 i art. 16 ustawy z  dnia 24  stycznia 
1991  r. o kombatantac

In [11]:
print(full_text)

©Kancelaria Sejmu     s. 1/7 
 
2023 -08-03 
  
Dz. U. 2020 poz. 321   
 
 
U S T AWA  
z dnia 9  stycznia 2020  r. 
o dodatkowym rocznym świadczeniu pieniężnym dla emerytów i  rencistów  
Art. 1. Ustawa określa warunki nabywania prawa do dodatkowego rocznego 
świadczenia pieniężnego oraz zasady jego wypłaty.  
Art. 2. 1. Dodatkowe roczne świadczenie pieniężne przysługuje osobom, 
które w  dniu 31  marca roku, w  którym wyp łacane jest dodatkowe roczne 
świadczenie pieniężne, mają prawo do:  
1) świadczeń, o  których mowa w  art. 2 pkt 1 lit. a i b ustawy z  dnia 29  maja 
1974  r. o zaopatrzeniu inwalidów wojennych i  wojskowyc h oraz ich rodzin 
(Dz. U. z 2022  r. poz.  2287),  
2) świadcz eń z  ubezpieczenia emerytalno -rentowego, o  których mowa 
w art. 18 pkt 1–4 ustawy z  dnia 20  grudnia 1990  r. o  ubezpieczeniu 
społecznym rolników (Dz.  U. z 2022 r. poz. 933, 1155 i 2140),  
3) świadczeń, o  których mowa w  art. 12 i art. 16 ustawy z  dnia 24  stycznia 
1991  r. o kombatantac

In [1]:
print(pages)

NameError: name 'pages' is not defined

In [13]:
full_text = cleaned_legal_act

In [14]:
full_text

'U S T AWA   z dnia 9  stycznia 2020  r.  o dodatkowym rocznym świadczeniu pieniężnym dla emerytów i  rencistów   Art. 1. Ustawa określa warunki nabywania prawa do dodatkowego rocznego  świadczenia pieniężnego oraz zasady jego wypłaty.   Art. 2. 1. Dodatkowe roczne świadczenie pieniężne przysługuje osobom,  które w  dniu 31  marca roku, w  którym wyp łacane jest dodatkowe roczne  świadczenie pieniężne, mają prawo do:   1) świadczeń, o  których mowa w  art. 2 pkt 1 lit. a i b ustawy z  dnia 29  maja  1974  r. o zaopatrzeniu inwalidów wojennych i  wojskowyc h oraz ich rodzin  (Dz. U. z 2022  r. poz.  2287),   2) świadcz eń z  ubezpieczenia emerytalno -rentowego, o  których mowa  w art. 18 pkt 1–4 ustawy z  dnia 20  grudnia 1990  r. o  ubezpieczeniu  społecznym rolników (Dz.  U. z 2022 r. poz. 933, 1155 i 2140),   3) świadczeń, o  których mowa w  art. 12 i art. 16 ustawy z  dnia 24  stycznia  1991  r. o kombatantach oraz niektórych osobach będących ofiarami represji  wojennych i  okresu p

In [15]:
import re

class CustomMarkdownSplitter:
    def __init__(self, patterns):
        self.patterns = patterns
        self.article_split_pattern = r'(Art\. \d+[a-zA-Z]?[\s\S]*?)(?=Art\. \d+[a-zA-Z]?|$)'
        self.passage_pattern = r'(?<!Art\.\s)(?<!Art\.\s\^d\.\s)(?<!Art\.\s\d)(?<!Art\.\s\d\d)(?<!Art\.\s\d\d\d)(?=\d+\.\s\w)'

    def split_text(self, text):
        splits = [text]
        article_split = re.findall(self.article_split_pattern, text)

        passage_split = [re.split(self.passage_pattern, article) for article in article_split  ]

        splits = new_splits
        return splits

# Define the patterns in the order you want to split
patterns = [
    r'(Art\. \d+[a-zA-Z]?[\s\S]*?)(?=Art\. \d+[a-zA-Z]?|$)',
    # r'(^Art\. \d+[a-zA-Z]?(?:–\d+)?\.)(.)+\w',  # Matches "Art. X", "Art. Xa", "Art. X-Y", etc.
    r'(\d+\.\s)',  # Matches "1. " 
    r'(\d+\)\s)',  # Matches "1) "
    r'(\([a-z]\)\s)'  # Matches "(a) "
]

# Initialize the custom splitter
custom_splitter = CustomMarkdownSplitter(patterns)

custom_splitter.split_text(full_text)

NameError: name 'new_splits' is not defined

In [23]:
import re

text = "Art. 12. 1. Dla celów podatku  dochodowego od osób fizycznych dodatkowe \nroczne świadczenie pieniężne stanowi świadczenie, o  którym mowa w  art. 12 ust. 7 \nustawy z  dnia 26  lipca 1991  r. o podatku dochodowym od osób fizycznych (Dz. U. \nz 2021 r. poz. 1128, z późn. zm.2)). \n[2. Do kwoty doda tkowego rocznego świadczenia pieniężnego stosuje się \nodpowiednio art.  66 ust. 1 pkt 16, art. 81 ust.  8 pkt 2 i art. 85 ust. 9–11 ustawy \nz dnia 27  sierpnia 2004  r. o świadczeniach opieki zdrowotnej finansowanych \nze środków publicznych (Dz.  U. z 2021  r. poz.  1285, z późn. zm.3)).]  \n<2. Do kwoty dodatkowego rocznego świadczenia pieniężnego stosuje się \nodpowiednio art.  66 ust. 1 pkt 16, art.  81 ust. 8 pkt 2, art. 83 ust. 1 \ni art. 85 ust. 9–11 ustawy z  dnia 27  sierpnia 2004  r. o świadczeniach opieki \n                                                           \n1) Zmiany tekstu jednolitego wymienionej ustawy zostały ogłoszone w Dz. U. z 2021 r. poz. 2270 \noraz z 2022 r. poz. 1, 66, 1079, 1692, 1700, 1812, 1967 , 2127 i 2140 . \n2) Zmiany tekstu jednolitego wymienionej ustawy zostały ogłoszone w Dz. U. z 2021 r. poz. 1163, \n1243, 1551, 1574, 1834, 1981, 2071, 2105, 2133, 2232, 2269, 2270, 2328, 2376, 2427, 2430 i \n2490 oraz z 2022 r. poz. 1, 24, 64, 138, 501, 558, 583, 646, 655, 830, 872, 1079, 1265, 1301, \n1358, 1459, 1512, 1561, 1692, 1846, 1967, 2014, 2127 i 2180 . \n3) Zmiany tekstu jednolitego wymienionej ustawy zostały ogłoszone w Dz. U. z 2021 r. poz. 1292, \n1559, 1773, 1834, 1981, 2105, 2120, 2232, 2270, 2427 i 2469 oraz z 2022 r. poz. 64, 91, 526, \n583, 65 5, 807, 974, 1002, 1079, 1265, 1352 , 1700, 1855, 2140 i 2180.  Nowe brzmienie \nust. 2 w art. 12 \nwejdzie w życie z \ndn. 1.01.2024 r. \n(Dz. U. z 2023 r. \npoz. 1407). "

# Replace valid split points with a unique delimiter
modified_text = re.sub(r'(?<!Art\. \d+\.\s)(?<=\d\.\s)', '###SPLIT###', text)

# Split the text at the delimiter
split_text = modified_text.split('###SPLIT###')

# Displaying the split text
split_text


In [10]:
!pip install PyPDF2


14069.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.0 MB/s eta 0:00:00


In [12]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_path)
    text_by_page = []

    # Extract text from each page
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        text_by_page.append(text)
    
    return text_by_page

def identify_footer(text_by_page):
    footer_candidates = {}
    
    # Check the last few lines of each page to find repetitive text patterns
    for page_text in text_by_page:
        lines = page_text.split('\n')
        if len(lines) > 5:  # Ensure there are enough lines to check
            footer_line = lines[-1].strip()  # Get the last line
            if footer_line in footer_candidates:
                footer_candidates[footer_line] += 1
            else:
                footer_candidates[footer_line] = 1
    
    # Identify the most common footer line
    footer_line, count = max(footer_candidates.items(), key=lambda item: item[1])
    
    # Assume footer is the line that appears on most pages
    if count > 1:
        return footer_line
    return None

def remove_footer(text_by_page, footer_line):
    cleaned_text_by_page = []
    
    for page_text in text_by_page:
        cleaned_text = page_text.replace(footer_line, '').strip()
        cleaned_text_by_page.append(cleaned_text)
    
    return cleaned_text_by_page

def main(pdf_path):
    text_by_page = extract_text_from_pdf(pdf_path)
    footer_line = identify_footer(text_by_page)
    
    if footer_line:
        print(f"Identified footer: '{footer_line}'")
        cleaned_text_by_page = remove_footer(text_by_page, footer_line)
    else:
        print("No repetitive footer found.")
        cleaned_text_by_page = text_by_page

    # Print cleaned text for each page
    for page_num, text in enumerate(cleaned_text_by_page):
        print(f"Page {page_num + 1}:\n{text}\n")

if __name__ == "__main__":
    pdf_path = "D19910024Lj.pdf"
    main(pdf_path)


Identified footer: 'poz. 1429).'
Page 1:
©Kancelaria Sejmu     s. 1/75 
   
2023 -08-10 
  
Dz. U. 1991 Nr 7 poz. 24   
 
 
U S T AWA  
z dnia 20  grudnia 1990  r. 
o ubezpieczeniu społecznym rolników1) 
Rozdział 1 
Przepisy ogólne  
Art. 1. 1. Ubezpieczenie społeczne rolników, zwane dalej „ubezpieczeniem”, 
obejmuje, na zasadach określonych w  ustawie, rolników i  pracujących z  nimi 
domowników oraz pomocników rolnika, którzy:  
1) posiadaj ą obywatelstwo polskie lub  
2) są uprawnieni do wykonywania pracy na terytorium Rzeczypospolitej Polskiej 
na podstawie art.  87 ustawy z  dnia 20  kwietnia 2004  r. o promocji zatrudnienia 
i instytucjach rynku pracy (Dz.  U. z  2022  r. poz.  690, z późn. zm.2)) lub są 
zwolnieni na podstawie przep isów szczególnych z  obowiązku posiadania 
zezwolenia na pracę.  
1a. (uchylony)  
2. W ubezpieczeniu wyodrębnia się:  
1) ubezpieczenie wypadkowe, chorobowe i  macierzyńskie;  
2) ubezpieczenie emerytalno -rentowe.  
Art. 2. 1. Ubezpieczenie realizu